In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import warnings
import os
os.chdir("../")
warnings.filterwarnings('ignore')
# from reportlab.lib.pagesizes import A4, landscape
# from reportlab.lib import colors
# from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
# from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
# from reportlab.lib.units import inch

In [2]:
df_2002 = pd.read_csv('data/final_csv/2002.csv')
df_2012 = pd.read_csv('data/final_csv/2012.csv')
df_2022 = pd.read_csv('data/final_csv/2022.csv')

In [3]:
df_2002["HHCHILDR"] = pd.to_numeric(df_2002["HHCHILDR"], errors="coerce").fillna(0)

In [4]:
df_2002["HHCHILDR"].unique()

array([ 0.,  2.,  1.,  3.,  4.,  5.,  6.,  8.,  7., 10.,  9.])

In [5]:
df_2012.columns

Index(['Unnamed: 0', 'V5_egal', 'V6_egal', 'V7_egal', 'V8_egal', 'V9_egal',
       'V11_egal', 'V5_egal_z', 'V6_egal_z', 'V7_egal_z', 'V8_egal_z',
       'V9_egal_z', 'V11_egal_z', 'eg_score', 'sex', 'hh_wrk_hrs', 'wrk_hrs',
       'country', 'age', 'age_bin', 'high_egal', 'low_housework',
       'highest_education', 'educ_4', 'educ_4_label', 'code_income_control',
       'code_higher_income', 'marital', 'work_status_std',
       'spouse_work_status_std', 'work_status', 'spouse_work_status',
       'urban_rural', 'eg_score_norm', 'TOPBOT', 'SPWRKHRS', 'C_ALPHAN',
       'LIVWOMAR', 'WWYKS', 'WWYKUS', 'SP_DEGREE', 'MOMORFAF', 'MEWH',
       'HHTODD', 'HHCHILDR', 'HHADULT', 'FAM_DIF', 'SHARE_HH', 'HW_FULFIL',
       'WO_WANT', 'WW_FAM_SUFFER', 'WW_CHILD_SUFFER', 'WW_WARM', 'DIV_HH_COOK',
       'DIV_HH_CLEAN', 'DIV_HH_GROC', 'DIV_HH_CARE', 'DIV_HH_LAUND',
       'SP_HH_FAM', 'SP_HH', 'LIFE_HAP', 'DIFF_CONC_WORK', 'HH_TIRED',
       'HH_FAM', 'WORK_TIRED', 'HH_WEEKEND', 'COHAB', 'HOMPOP',

In [6]:
def normalize_column(series):
    """
    Normalize a numeric series to 0-1 range using min-max scaling.
    """
    return (series - series.min()) / (series.max() - series.min())

# Applying to eg_score columns
df_2002['eg_score_norm'] = normalize_column(df_2002['eg_score'])
df_2012['eg_score_norm'] = normalize_column(df_2012['eg_score'])
df_2022['eg_score_norm'] = normalize_column(df_2022['eg_score'])

In [7]:
df_2002 = df_2002.copy()
df_2012 = df_2012.copy()
df_2022 = df_2022.copy()

# Adding a 'year' column to see from which year each row comes
df_2002['year'] = 2002
df_2012['year'] = 2012
df_2022['year'] = 2022

# Combining all three datasets
combined_df = pd.concat([df_2002, df_2012, df_2022], ignore_index=True)


combined_df = combined_df.drop_duplicates(subset=combined_df.columns.difference(['year']))

print(f"Total rows after merging: {len(combined_df)}")
print(f"\nRows per year:")
print(combined_df['year'].value_counts().sort_index())

print(f"\nDataframe info:")
print(combined_df.info())
os.makedirs('./data/extras', exist_ok=True)
combined_df.to_csv('./data/extras/combined_dataset_new.csv', index=False)

Total rows after merging: 154154

Rows per year:
year
2002    46638
2012    61754
2022    45762
Name: count, dtype: int64

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154154 entries, 0 to 154153
Data columns (total 89 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              154154 non-null  int64  
 1   v4_egal                 89226 non-null   float64
 2   v5_egal                 88057 non-null   float64
 3   v6_egal                 90165 non-null   float64
 4   v7_egal                 44089 non-null   float64
 5   v8_egal                 43720 non-null   float64
 6   v11_egal                45475 non-null   float64
 7   v4_egal_z               89226 non-null   float64
 8   v5_egal_z               88057 non-null   float64
 9   v6_egal_z               90165 non-null   float64
 10  v7_egal_z               44089 non-null   float64
 11  v8_egal_z               43720 non-null   fl

In [8]:
combined_df.head()

,Unnamed: 0,v4_egal,v5_egal,v6_egal,v7_egal,v8_egal,v11_egal,v4_egal_z,v5_egal_z,v6_egal_z,...,V7_egal_z,V8_egal_z,V9_egal_z,V11_egal_z,v1_egal,v2_egal,v3_egal,v1_egal_z,v2_egal_z,v3_egal_z
0,0,2.0,2.0,2.0,3.0,3.0,3.0,-1.357029,-0.614036,-0.676980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5.0,5.0,5.0,5.0,1.0,5.0,1.105198,1.788120,1.680397,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,4.0,3.0,2.0,3.0,2.0,4.0,0.284456,0.186683,-0.676980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2.0,2.0,1.0,4.0,4.0,4.0,-1.357029,-0.614036,-1.462772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3.0,2.0,2.0,3.0,2.0,3.0,-0.536287,-0.614036,-0.676980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
combined_df['WO_WANT'].unique()

array(['Neither agree nor disagree', 'Strongly disagree', 'Disagree',
       'Agree', 'Strongly agree', nan], dtype=object)

In [10]:
df_analysis = combined_df.copy()

In [11]:
df_analysis['HHCHILDR'].unique()

array([ 0.,  2.,  1.,  3.,  4.,  5.,  6.,  8.,  7., 10.,  9., nan, 21.,
       18., 12., 15., 14., 11.])

MAPPING OF ALL CATEGORICAL VALUES

In [12]:
opinion_map = {
    'Strongly disagree': 1,
    'Disagree': 2,
    'Neither agree nor disagree': 3,
    'Agree': 4,
    'Strongly agree': 5
}


employment_mapping = {
    'Paid work': 'Employed',
    'Apprentice/Trainee': 'Employed',
    'Education': 'Employed',
    'Military/Community service': 'Employed',
    'Unemployed': 'Unemployed',
    'Retired': 'Unemployed',
    'Domestic work': 'Unemployed',
    'Sick/Disabled': 'Unemployed',
    'Other': 'Unemployed',
    'DK/No answer': 'Unemployed'
}


spouse_employment_map = {
    "Paid work": "Employed",
    "Apprentice/Trainee": "Employed",
    "Education": "Employed",
    "Unemployed": "Unemployed",
    "Domestic work": "Unemployed",
    "Retired": "Unemployed",
    "Sick/Disabled": "Unemployed",
    "Help family member": "Unemployed",
    "Military/Community service": "Unemployed",
    "Other": "Unemployed",
    "DK/No answer": "Unemployed",
    "NAP": "Unemployed"
}


momorfaf_mapping = {
    'Fathers much better': -2,
    'Fathers somewhat better': -1,
    'Equally suited': 0,
    'Mothers somewhat better': 1,
    'Mothers much better': 2
}

work_status_mapping = {
'Paid work': 1,
    'Apprentice/Trainee': 1,
    'Education': 1,
    'Military/Community service': 1,
    'Unemployed': 0,
    'Retired': 0,
    'Domestic work': 0,
    'Sick/Disabled': 0,
    'Other': 0,
    'DK/No answer': 0
}

spouse_work_status_mapping = {
    'Paid work': 1,
    'Apprentice/Trainee': 1,
    'Education': 1,
    'Military/Community service': 1,
    'Retired': 0,
    'Domestic work': 0,
    'Unemployed': 0,
    'Sick/Disabled': 0,
    'Help family member': 0,
    'Other': 0,
    'DK/No answer': 0,
    'NAP': 0  
}

div_hh_care_mapping = {
    'Always partner': -2,
    'Usually partner': -1,
    'About equal': 0,
    'Usually respondent': 1,
    'Always respondent': 2,
    'Third person': None,  
    'NAP': None
}

sex_mapping = {
    'Male' : 0,
    'Female' : 1
}

edu_map = {
    "No/Primary": 1,
    "Secondary": 2,
    "Post-sec / Short tertiary": 3,
    "University+": 4
}

warm_opinion_map = {
    'Strongly disagree': 5,
    'Disagree': 4,
    'Neither agree nor disagree': 3,
    'Agree': 2,
    'Strongly agree': 1
}

Mapping all the ordinal/binary values to the variables

In [13]:
df_analysis['WW_CHILD_SUFFER'] = df_analysis['WW_CHILD_SUFFER'].map(opinion_map)
df_analysis['WW_FAM_SUFFER'] = df_analysis['WW_FAM_SUFFER'].map(opinion_map)
df_analysis['WO_WANT'] = df_analysis['WO_WANT'].map(opinion_map)
df_analysis['HW_FULFIL'] = df_analysis['HW_FULFIL'].map(opinion_map)
df_analysis['MEWH'] = df_analysis['MEWH'].map(opinion_map)
df_analysis['MEWH'] = df_analysis['MEWH'].map(opinion_map)
df_analysis['WW_WARM'] = df_analysis['WW_WARM'].map(warm_opinion_map)

In [14]:
df_analysis['MOMORFAF'] = df_analysis['MOMORFAF'].map(momorfaf_mapping)
df_analysis['DIV_HH_CARE'] = df_analysis['DIV_HH_CARE'].map(div_hh_care_mapping)
df_analysis['sex'] = df_analysis['sex'].map(sex_mapping)
df_analysis['work_status_std'] = df_analysis['work_status_std'].map(work_status_mapping)
df_analysis['spouse_work_status_std'] = df_analysis['spouse_work_status_std'].map(spouse_work_status_mapping)

In [15]:
df_analysis['educ_4_encoded'] = df_analysis['educ_4_label'].map(edu_map)

In [16]:
# Dependent variables
PILLARS = ['WW_WARM', 'WW_CHILD_SUFFER', 'WW_FAM_SUFFER', 'WO_WANT', 'HW_FULFIL', 'MEWH']
ALL_OUTCOMES = ['eg_score_norm'] + PILLARS
# Controls (for testing both the hypotheses)
CONTROLS = ['age', 'work_status_std', 'spouse_work_status_std', 'educ_4_encoded']
# This variable is the centered year variable
df_analysis['year_c'] = df_analysis['year'] - 2002
# Splitting by gender
df_male = df_analysis[df_analysis['sex'] == 0].copy()
df_female = df_analysis[df_analysis['sex'] == 1].copy()
print(f"Total respondents: {len(df_analysis)}")
print(f"Males: {len(df_male)}, Females: {len(df_female)}")

Total respondents: 154154
Males: 70134, Females: 83776


In [17]:
# We are calculating sample mean of each control variable per year
def get_sample_means(df, controls, years=[0, 10, 20]):
    means = {}
    for y in years:
        subset = df[df['year_c'] == y]
        means[y] = {ctrl: subset[ctrl].mean() for ctrl in controls}
    return means
# OLS Regression
def run_regression(df, outcome, focus_var, controls, interaction=True):
    ctrl_str = ' + '.join(controls)
    if interaction:
        formula = f"{outcome} ~ {focus_var} + year_c + {focus_var}:year_c + {ctrl_str}"
    else:
        formula = f"{outcome} ~ {focus_var} + year_c + {ctrl_str}"
    vars_in_model = [outcome, focus_var, 'year_c'] + controls
    df_clean = df[vars_in_model].dropna()
    model = smf.ols(formula, data=df_clean).fit(cov_type='HC1')
    return model

# function for prediction
def predict_scores(model, focus_var, focus_values, sample_means, controls):
    predictions = []
    for year_c in [0, 10, 20]:  
        for val in focus_values:
            row = {
                focus_var: val,
                'year_c': year_c
            }
            for ctrl in controls:
                row[ctrl] = sample_means[year_c][ctrl]
            
            pred = model.predict(pd.DataFrame([row]))[0]
            predictions.append({
                'Year': 2002 + year_c,
                focus_var: val,
                'Predicted_Score': round(pred, 4)
            })
    return pd.DataFrame(predictions)
def extract_results(model, focus_var):
    results = {}
    # Key terms to extract
    terms = [focus_var, 'year_c', f'{focus_var}:year_c']
    for term in terms:
        if term in model.params.index:
            coef = model.params[term]
            pval = model.pvalues[term]
            ci = model.conf_int().loc[term]
            
            results[term] = {
                'Coefficient': round(coef, 4),
                'p_value': pval,
                'CI_lower': round(ci[0], 4),
                'CI_upper': round(ci[1], 4),
            }
    results['R_squared'] = round(model.rsquared, 4)
    results['N'] = int(model.nobs)
    return results

In [18]:
print(df_analysis['work_status_std'].unique())
print(df_analysis['spouse_work_status_std'].unique())
print(df_analysis['educ_4_encoded'].unique())

[1 0]
[0 1]
[ 3.  4.  2.  1. nan]


MAIN CODE

In [19]:
# Dependent variables
PILLARS = ['WW_WARM', 'WW_CHILD_SUFFER', 'WW_FAM_SUFFER', 'WO_WANT', 'HW_FULFIL', 'MEWH']
ALL_OUTCOMES = ['eg_score_norm'] + PILLARS
# Controls that will be used in the regression models
CONTROLS = ['age', 'work_status_std', 'spouse_work_status_std', 'educ_4_encoded']
# Sex mapping
SEX_FEMALE = 1
SEX_MALE   = 0
# # This variable is the centered year variable
df_analysis['year'] = pd.to_numeric(df_analysis['year'], errors='coerce')
df_analysis['year_c'] = df_analysis['year'] - 2002
df_analysis['kids_cat'] = pd.to_numeric(df_analysis['HHCHILDR'], errors='coerce').clip(upper=3)
# Splitting by gender
df_male   = df_analysis[df_analysis['sex'] == SEX_MALE].copy()
df_female = df_analysis[df_analysis['sex'] == SEX_FEMALE].copy()
print(f"Total respondents: {len(df_analysis):,}")
print(f"Males: {len(df_male):,}, Females: {len(df_female):,}")

Total respondents: 154,154
Males: 70,134, Females: 83,776


In [20]:
# Returns mode for categorical or mean for continuous data.
def _mode_or_mean(series: pd.Series, treat_as_categorical: bool):
    s = series.dropna()
    if s.empty:
        return np.nan
    if treat_as_categorical:
        m = s.mode()
        return m.iloc[0] if not m.empty else s.iloc[0]
    return s.mean()

def get_control_values_by_year(df, controls, years=(0, 10, 20), continuous_controls=('age', 'educ_4_encoded')):
    out = {}
    for y in years:
        subset = df[df['year_c'] == y]
        out[y] = {}
        for ctrl in controls:
            treat_as_cat = (ctrl not in continuous_controls)
            out[y][ctrl] = _mode_or_mean(subset[ctrl], treat_as_cat)
    return out

def run_regression(df, outcome, focus_var, controls, interaction=True):
    ctrl_str = ' + '.join(controls)
    if interaction:
        formula = f"{outcome} ~ {focus_var} + year_c + {focus_var}:year_c + {ctrl_str}"
    else:
        formula = f"{outcome} ~ {focus_var} + year_c + {ctrl_str}"

    vars_in_model = [outcome, focus_var, 'year_c', 'year'] + controls
    df_clean = df[vars_in_model].dropna()

    if df_clean['year'].nunique() >= 2:
        model = smf.ols(formula, data=df_clean).fit(
            cov_type='cluster',
            cov_kwds={'groups': df_clean['year']}
        )
    else:
        model = smf.ols(formula, data=df_clean).fit(cov_type='HC1')

    return model, df_clean

def predict_scores(model, focus_var, focus_values, control_values_by_year, controls, years=(0,10,20)):
    preds = []
    for year_c in years:
        for val in focus_values:
            row = {focus_var: val, 'year_c': year_c}
            for ctrl in controls:
                row[ctrl] = control_values_by_year[year_c][ctrl]

            pred = float(model.predict(pd.DataFrame([row]))[0])
            preds.append({
                'Year': int(2002 + year_c),
                focus_var: val,
                'Predicted_Score': round(pred, 4)
            })
    return pd.DataFrame(preds)

def extract_results(model, focus_var):
    results = {}
    terms = [focus_var, 'year_c', f'{focus_var}:year_c']
    for term in terms:
        if term in model.params.index:
            coef = float(model.params[term])
            pval = float(model.pvalues[term])
            ci = model.conf_int().loc[term]
            if pval < 0.001:
                stars = '***'
            elif pval < 0.01:
                stars = '**'
            elif pval < 0.05:
                stars = '*'
            else:
                stars = ''
            results[term] = {
                'Coefficient': round(coef, 4),
                'p_value': pval,
                'CI_lower': round(float(ci[0]), 4),
                'CI_upper': round(float(ci[1]), 4),
                'stars': stars,
                'display': f"{round(coef, 4)}{stars}"
            }
    results['R_squared'] = round(float(model.rsquared), 4)
    results['N'] = int(model.nobs)
    return results

In [21]:
print("CONTROLS:", CONTROLS)
print("\ndf_male columns:", df_male.columns.tolist())
print("\nNaN counts:\n", df_male[['kids_cat'] + list(CONTROLS)].isna().sum())

CONTROLS: ['age', 'work_status_std', 'spouse_work_status_std', 'educ_4_encoded']

df_male columns: ['Unnamed: 0', 'v4_egal', 'v5_egal', 'v6_egal', 'v7_egal', 'v8_egal', 'v11_egal', 'v4_egal_z', 'v5_egal_z', 'v6_egal_z', 'v7_egal_z', 'v8_egal_z', 'v11_egal_z', 'eg_score', 'sex', 'hh_wrk_hrs', 'wrk_hrs', 'country', 'age', 'age_bin', 'high_egal', 'low_housework', 'highest_education', 'educ_4', 'educ_4_label', 'code_income_control', 'code_higher_income', 'marital', 'work_status_std', 'spouse_work_status_std', 'work_status', 'spouse_work_status', 'urban_rural', 'eg_score_norm', 'TOPBOT', 'SPWRKHRS', 'C_ALPHAN', 'LIVWOMAR', 'WWYKS', 'WWYKUS', 'SP_DEGREE', 'MOMORFAF', 'MEWH', 'HHTODD', 'HHCHILDR', 'HHADULT', 'FAM_DIF', 'SHARE_HH', 'HW_FULFIL', 'WO_WANT', 'WW_FAM_SUFFER', 'WW_CHILD_SUFFER', 'WW_WARM', 'DIV_HH_COOK', 'DIV_HH_CLEAN', 'DIV_HH_GROC', 'DIV_HH_CARE', 'DIV_HH_LAUND', 'SP_HH_FAM', 'SP_HH', 'LIFE_HAP', 'DIFF_CONC_WORK', 'HH_TIRED', 'HH_FAM', 'WORK_TIRED', 'HH_WEEKEND', 'COHAB', 'HOMPOP

HYPOTHESIS TESTING

In [22]:
def test_hypothesis(results, focus_var, hypothesis_type, outcome):
    STANDARD_SCALE = ['eg_score_norm', 'WW_WARM']
    is_standard = outcome in STANDARD_SCALE
    interaction_key = f"{focus_var}:year_c"
    main_coef = results[focus_var]['Coefficient']
    main_pval = results[focus_var]['p_value']
    int_coef  = results[interaction_key]['Coefficient']
    int_pval  = results[interaction_key]['p_value']
    if hypothesis_type == 'H1':
        if is_standard:
            main_supported = (main_coef < 0) and (main_pval < 0.05)
            int_supported  = (int_coef  > 0) and (int_pval  < 0.05)
        else:
            main_supported = (main_coef > 0) and (main_pval < 0.05)
            int_supported  = (int_coef  < 0) and (int_pval  < 0.05)
    else:  # Hypothesis 2
        if is_standard:
            main_supported = (main_coef > 0) and (main_pval < 0.05)
            int_supported  = (int_coef  > 0) and (int_pval  < 0.05)
        else:
            main_supported = (main_coef < 0) and (main_pval < 0.05)
            int_supported  = (int_coef  < 0) and (int_pval  < 0.05)
    overall = (
        "Supported" if (main_supported and int_supported) else
        "Partially Supported" if (main_supported or int_supported) else
        "Not Supported"
    )
    return {
        'main_supported': main_supported,
        'int_supported': int_supported,
        'overall': overall,
        'main_coef': main_coef,
        'main_pval': main_pval,
        'int_coef': int_coef,
        'int_pval': int_pval
    }

SPEARMAN CORRELATION FOR CHECKING THE EFFECT OF NO. OF KIDS IN THE FAMILY

In [23]:
import pandas as pd
from scipy.stats import spearmanr

SEX_COL = "sex"         
KIDS_COL = "HHCHILDR"
OUTCOMES = ["WW_FAM_SUFFER", "WW_CHILD_SUFFER", "WW_WARM"]
USE_TOPCODED_KIDS = False

df = df_analysis.copy()

if USE_TOPCODED_KIDS:
    df["kids_cat"] = pd.to_numeric(df[KIDS_COL], errors="coerce").clip(upper=3)
    kids_var = "kids_cat"
else:
    df[KIDS_COL] = pd.to_numeric(df[KIDS_COL], errors="coerce")
    kids_var = KIDS_COL


# speaman correlation table
def spearman_table(df_sub, kids_var, outcomes):
    rows = []
    for y in outcomes:
        tmp = df_sub[[kids_var, y]].dropna()
        if len(tmp) < 10:
            rows.append({"Outcome": y, "N": len(tmp), "rho": None, "p_value": None})
            continue
        rho, p = spearmanr(tmp[kids_var], tmp[y])
        rows.append({"Outcome": y, "N": len(tmp), "rho": rho, "p_value": p})
    return pd.DataFrame(rows)
results = {}

for sex_value, label in [(0, "Female"), (1, "Male")]:
    df_sub = df[df[SEX_COL] == sex_value].copy()
    print(f"Spearman correlations | {label} (sex={sex_value}) | kids = {kids_var}")
    tab = spearman_table(df_sub, kids_var, OUTCOMES)
    tab["rho"] = tab["rho"].round(4)
    tab["p_value"] = tab["p_value"].apply(lambda x: f"{x:.4g}" if pd.notnull(x) else None)
    print(tab.to_string(index=False))
    results[sex_value] = tab

Spearman correlations | Female (sex=0) | kids = HHCHILDR
        Outcome     N    rho   p_value
  WW_FAM_SUFFER 64744 0.0343 2.547e-18
WW_CHILD_SUFFER 64498 0.0228 7.324e-09
        WW_WARM 64942 0.0018    0.6392
Spearman correlations | Male (sex=1) | kids = HHCHILDR
        Outcome     N    rho   p_value
  WW_FAM_SUFFER 77856 0.0531 1.059e-49
WW_CHILD_SUFFER 77775 0.0335 1.038e-20
        WW_WARM 78418 0.0070   0.04888


In [24]:
df = df_analysis.copy()
df["HHCHILDR"] = pd.to_numeric(df["HHCHILDR"], errors="coerce")
df["HHTODD"]   = pd.to_numeric(df["HHTODD"], errors="coerce")
df[["HHCHILDR", "HHTODD"]] = df[["HHCHILDR", "HHTODD"]].fillna(0)
df["TOTAL_KIDS"] = df["HHCHILDR"] + df["HHTODD"]

In [25]:
SEX_COL = "sex"
KIDS_COL = "TOTAL_KIDS"

OUTCOMES = ["WW_FAM_SUFFER", "WW_CHILD_SUFFER", "WW_WARM"]

def spearman_table(df_sub, kids_var, outcomes):
    rows = []
    for y in outcomes:
        tmp = df_sub[[kids_var, y]].dropna()
        if len(tmp) < 10:
            rows.append({"Outcome": y, "N": len(tmp), "rho": None, "p_value": None})
            continue
        rho, p = spearmanr(tmp[kids_var], tmp[y])
        rows.append({
            "Outcome": y,
            "N": len(tmp),
            "rho": round(rho, 4),
            "p_value": f"{p:.4g}"
        })
    return pd.DataFrame(rows)

# Separate matrices based on gender
for sex_value, label in [(0, "Female"), (1, "Male")]:
    df_sub = df[df[SEX_COL] == sex_value].copy()
    print(f"Spearman correlations | {label} | TOTAL_KIDS")
    res = spearman_table(df_sub, KIDS_COL, OUTCOMES)
    print(res.to_string(index=False))

Spearman correlations | Female | TOTAL_KIDS
        Outcome     N    rho   p_value
  WW_FAM_SUFFER 66927 0.0377 1.894e-22
WW_CHILD_SUFFER 66686 0.0193 5.944e-07
        WW_WARM 67157 0.0037    0.3313
Spearman correlations | Male | TOTAL_KIDS
        Outcome     N    rho   p_value
  WW_FAM_SUFFER 80383 0.0673 1.904e-81
WW_CHILD_SUFFER 80293 0.0381 3.272e-27
        WW_WARM 80965 0.0187 1.056e-07
